<a href="https://colab.research.google.com/github/gtbnhyujmj/-Good-Auto_Shorts_Maker/blob/main/%5BV3%5D_Stickered_Movie.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# === 1. 安裝套件 ===
!pip install ffmpeg-python opencv-python pillow tqdm pydub

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# === 2. 載入模組 ===
import os, shutil
import ffmpeg
import numpy as np
from PIL import Image
from tqdm import tqdm
from pydub import AudioSegment

# === 3. 工具 ===

In [8]:
def extract_audio(input_mov, output_wav):
    (
        ffmpeg
        .input(input_mov)
        .output(output_wav, acodec='pcm_s16le', ac=2, ar='44100')
        .run(overwrite_output=True)
    )
    print(f"✅ 分離音軌：{output_wav}")

In [9]:
def has_audio_track(video_path):
    """判斷影片檔是否有音軌"""
    import ffmpeg
    probe = ffmpeg.probe(video_path)
    return any(s for s in probe['streams'] if s['codec_type'] == 'audio')

In [10]:
def auto_clean_audio(input_wav, output_wav, silence_db_thresh=-45.0, min_chunk_ms=100):
    audio = AudioSegment.from_wav(input_wav)
    cleaned = AudioSegment.silent(duration=0, frame_rate=audio.frame_rate)
    total_len = len(audio)
    for i in range(0, total_len, min_chunk_ms):
        chunk = audio[i:i+min_chunk_ms]
        if chunk.dBFS < silence_db_thresh:
            cleaned += AudioSegment.silent(duration=min_chunk_ms, frame_rate=audio.frame_rate)
        else:
            cleaned += chunk
    cleaned.export(output_wav, format="wav")
    print(f"✅ 自動消噪完成：{output_wav}")

In [11]:
def align_and_mix_audio(audio_list, total_frames, fps, output_wav):
    total_length_ms = int(total_frames / fps * 1000)
    result = AudioSegment.silent(duration=total_length_ms)
    for audio_path in audio_list:
        if audio_path is None or not os.path.exists(audio_path):
            continue
        audio = AudioSegment.from_file(audio_path)
        repeats = int(np.ceil(total_length_ms / len(audio)))
        audio_full = (audio * repeats)[:total_length_ms]
        result = result.overlay(audio_full)
    result.export(output_wav, format="wav")
    print(f"✅ 多層音軌補齊並混音完成：{output_wav}")

In [12]:
def extract_frames_with_alpha(input_path, output_folder, fps=None, ref_img=None, ref_frames=None):
    os.makedirs(output_folder, exist_ok=True)
    if input_path.lower().endswith(('.mov', '.mp4')):
        options = {'pix_fmt': 'rgba'}
        if fps:
            options['vf'] = f'fps={fps}'
        (
            ffmpeg
            .input(input_path)
            .output(os.path.join(output_folder, 'frame_%05d.png'), **options)
            .run(overwrite_output=True)
        )
    elif os.path.isdir(input_path):
        png_files = [f for f in sorted(os.listdir(input_path)) if f.endswith('.png')]
        if len(png_files) == 0:
            raise ValueError(f"資料夾 {input_path} 裡面沒有 PNG！")
        for i, fname in enumerate(png_files):
            img = Image.open(os.path.join(input_path, fname)).convert("RGBA")
            img.save(os.path.join(output_folder, f"frame_{i+1:05d}.png"))
    elif input_path.lower().endswith('.png'):
        img = Image.open(input_path).convert("RGBA")
        img.save(os.path.join(output_folder, "frame_00001.png"))
    elif input_path.lower().endswith(('.mp3', '.wav')):
        # 若沒參考圖，預設1080x1920
        if ref_img is not None and ref_frames is not None:
            size = ref_img.size
            nframes = ref_frames
        else:
            size = (1080, 1920)
            nframes = 1
        for i in range(nframes):
            transparent = Image.new("RGBA", size, (0,0,0,0))
            transparent.save(os.path.join(output_folder, f"frame_{i+1:05d}.png"))
    else:
        raise ValueError("input_path 必須是 MOV、MP4、PNG 資料夾、單張 PNG 或 MP3/WAV")
    print(f"✅ {input_path} 已轉換為 PNG 幀：{output_folder}")

In [13]:
from PIL import Image
import os

def blend_frames_multi(bg_folder, fg_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    bg_frames = sorted(os.listdir(bg_folder))
    fg_frames = sorted(os.listdir(fg_folder))
    bg_len = len(bg_frames)
    fg_len = len(fg_frames)
    total = max(bg_len, fg_len)

    for i in range(total):
        bg_img = Image.open(os.path.join(bg_folder, bg_frames[i % bg_len])).convert("RGBA")
        fg_img = Image.open(os.path.join(fg_folder, fg_frames[i % fg_len])).convert("RGBA")

        # 新增：把 fg resize 成跟 bg 一樣大
        if fg_img.size != bg_img.size:
            fg_img = fg_img.resize(bg_img.size, Image.LANCZOS)

        out_img = Image.alpha_composite(bg_img, fg_img)
        out_img.save(os.path.join(output_folder, f"frame_{i+1:05d}.png"))
    print(f"✅ 疊圖完成：{output_folder}")

In [14]:
def frames_to_mov_prores4444(frames_folder, output_path, fps=30):
    (
        ffmpeg
        .input(os.path.join(frames_folder, 'frame_%05d.png'), framerate=fps)
        .output(output_path, vcodec='prores_ks', profile=4, pix_fmt='yuva444p10le')
        .run(overwrite_output=True)
    )
    print(f"✅ MOV (ProRes 4444) 輸出完成（透明+壓縮）：{output_path}")

# === 4. 主流程 ===

In [15]:
def super_layered_composite(input_list, fps=30, silence_db_thresh=-45.0):
    import os, shutil
    from PIL import Image
    from tqdm import tqdm
    from pydub import AudioSegment
    import numpy as np

    prev_frames_dir = None
    audio_tracks = []
    max_frames = 0
    ref_img = None
    ref_frames = None

    # Step 1: 拆解所有層
    for idx, inp in enumerate(input_list):
        frames_dir = f"/content/layer_{idx+1}_frames"

        # 決定參考圖層（用第一個圖片/影片作基準）
        if ref_img is None and not inp.lower().endswith(('.mp3', '.wav')):
            extract_frames_with_alpha(inp, frames_dir, fps=fps)
            sample_file = os.path.join(frames_dir, sorted(os.listdir(frames_dir))[0])
            ref_img = Image.open(sample_file)
            ref_frames = len([f for f in os.listdir(frames_dir) if f.endswith('.png')])
        else:
            extract_frames_with_alpha(inp, frames_dir, fps=fps, ref_img=ref_img, ref_frames=ref_frames)

        n_frames = len([f for f in os.listdir(frames_dir) if f.endswith('.png')])
        max_frames = max(max_frames, n_frames)

        # Step 2: 音訊層處理
        is_video = inp.lower().endswith(('.mov', '.mp4'))
        is_mp3 = inp.lower().endswith('.mp3')
        is_wav = inp.lower().endswith('.wav')
        audio_wav = f"/content/layer_{idx+1}_audio.wav"

        if is_video and has_audio_track(inp):
            extract_audio(inp, audio_wav)
            auto_clean_audio(audio_wav, audio_wav, silence_db_thresh)
            audio_tracks.append(audio_wav)
        elif is_mp3:
            AudioSegment.from_mp3(inp).export(audio_wav, format="wav")
            auto_clean_audio(audio_wav, audio_wav, silence_db_thresh)
            audio_tracks.append(audio_wav)
        elif is_wav:
            auto_clean_audio(inp, audio_wav, silence_db_thresh)
            audio_tracks.append(audio_wav)
        else:
            audio_tracks.append(None)

        # Step 3: 疊圖
        if prev_frames_dir is None:
            prev_frames_dir = frames_dir
        else:
            out_dir = f"/content/merge_{idx}_frames"
            blend_frames_multi(prev_frames_dir, frames_dir, out_dir)
            shutil.rmtree(prev_frames_dir)
            shutil.rmtree(frames_dir)
            prev_frames_dir = out_dir

    # Step 4: 合成影片
    output_mov = '/content/final_output.mov'
    frames_to_mov_prores4444(prev_frames_dir, output_mov, fps=fps)

    # Step 5: 音訊補齊混音
    mix_audio_wav = "/content/final_output_audio.wav"
    align_and_mix_audio([a for a in audio_tracks if a], max_frames, fps, mix_audio_wav)

    # Step 6: mux
    output_mov_with_audio = '/content/final_output_with_audio.mov'
    import ffmpeg
    video_stream = ffmpeg.input(output_mov)
    audio_stream = ffmpeg.input(mix_audio_wav)
    (
        ffmpeg
        .output(video_stream, audio_stream, output_mov_with_audio, vcodec='copy', acodec='aac', shortest=None)
        .run(overwrite_output=True)
    )
    print(f"🎉 最終影片（含聲音）輸出：{output_mov_with_audio}")
    return output_mov_with_audio

# === 5. 用法 ===

In [16]:
input_list = [
    '/content/drive/MyDrive/shorts/backgrounds_MP4/bg2.mp4',  # 第一層影片/圖片
    '/content/drive/MyDrive/shorts/characters/char3.png',
    "/content/drive/MyDrive/shorts/PNG_Overlayers/OverLayers1.png",
    "/content/drive/MyDrive/shorts/Text_Overlayers/text_0.png",
    "/content/drive/MyDrive/shorts/BGM_MOV/final_output_with_audio.mov",
    # "",
    # 第二層
    # '/content/貼紙夾',        # 第三層可用資料夾
    # ... 無限層
]

In [17]:
final_mov = super_layered_composite(input_list, fps=30)

✅ /content/drive/MyDrive/shorts/backgrounds_MP4/bg2.mp4 已轉換為 PNG 幀：/content/layer_1_frames
✅ /content/drive/MyDrive/shorts/characters/char3.png 已轉換為 PNG 幀：/content/layer_2_frames
✅ 疊圖完成：/content/merge_1_frames
✅ /content/drive/MyDrive/shorts/PNG_Overlayers/OverLayers1.png 已轉換為 PNG 幀：/content/layer_3_frames
✅ 疊圖完成：/content/merge_2_frames
✅ /content/drive/MyDrive/shorts/Text_Overlayers/text_0.png 已轉換為 PNG 幀：/content/layer_4_frames
✅ 疊圖完成：/content/merge_3_frames
✅ /content/drive/MyDrive/shorts/BGM_MOV/final_output_with_audio.mov 已轉換為 PNG 幀：/content/layer_5_frames
✅ 分離音軌：/content/layer_5_audio.wav
✅ 自動消噪完成：/content/layer_5_audio.wav
✅ 疊圖完成：/content/merge_4_frames
✅ MOV (ProRes 4444) 輸出完成（透明+壓縮）：/content/final_output.mov
✅ 多層音軌補齊並混音完成：/content/final_output_audio.wav
🎉 最終影片（含聲音）輸出：/content/final_output_with_audio.mov


# MP3 >>> MOV

In [18]:
from pydub import AudioSegment
import ffmpeg

# 你想要的秒數
DURATION = 5  # 秒

# 圖片、音訊、影片路徑
IMAGE = '/content/drive/MyDrive/shorts/OK_Stamps/OK_1080_001.png'
AUDIO = '/content/drive/MyDrive/shorts/BGM_MP3/vine_boom.mp3'
OUTPUT = '/content/drive/MyDrive/shorts/BGM_MOV/final_output_with_audio.mov'

# 先產生5秒動畫（透明）
(
    ffmpeg
    .input(IMAGE, loop=1, t=DURATION, framerate=30)
    .output('/content/looped.mov', vcodec='prores_ks', profile=4, pix_fmt='yuva444p10le', framerate=30)
    .run(overwrite_output=True)
)

# 然後再合成音軌（截到5秒剛好）
video_stream = ffmpeg.input('/content/looped.mov')
audio_stream = ffmpeg.input(AUDIO)
(
    ffmpeg
    .output(video_stream, audio_stream, OUTPUT, vcodec='copy', acodec='aac', shortest=None)
    .run(overwrite_output=True)
)

print(f"✅ 成功產生5秒有聲音的影片：{OUTPUT}")

✅ 成功產生5秒有聲音的影片：/content/drive/MyDrive/shorts/BGM_MOV/final_output_with_audio.mov
